In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier

In [50]:
data = pd.read_csv(r'C:\Users\hp\Desktop\heart.csv')

#Filtering data by POSITIVE and NEGATIVE heart disease patient
#pos_data = data[data['target']==1]
#(pos_data.describe())

#pos_data = data[data['target']==0]
#pos_data.describe()

# calculating the correlation matrix to understand the relationship of features and targets
#corr = data.corr()
#plt.subplots(figsize=(15,10))
#sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,annot=True, cmap=sns.diverging_palette
 #          (220, 20, as_cmap=True))
#sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette
 #          (220, 20, as_cmap=True))


In [48]:
# Split the data into training and testing
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

# preprocessing our data with the standardscaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [49]:
#====================MY DEEP LEARNING MODEL=====================================#

# Creating a model with 4 dense layers (dense layers contain our neurons)
# when a layer is densely connected it means the layers are fully connected to each other
# The model uses BatchNormilzation() in order to scale our data in the range: [0, 1]
# i'm using the relu(rectified linear unit) optimizer and the sigmoid optimizer for the output layer
# the model is a sequential model
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', input_shape=[x.shape[1]]),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

#  I Compiled the model using adam optimizer
# I Measured the loss with binary_crossentropy another loss funtion i can use is categorical_crossentropy
# Set metrics to binary_accuracy, which lets us know how accurate the model is
# The accuracy lies on a scale of [0.0, 1.0],
# In order to convert this accuracy to percentage, move the decimal point to the right twice (acc*100)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Finally fit the model, training it with X_train and y_train, and validating it with X_test and y_test
# I Used 25 data points per epoch (the amount of epochs is the amount of times we feed the model data)
# In this case i'm using 3 epochs
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=25,
    epochs=3,
)


Epoch 1/3
33/33 [==============================] - 15s 75ms/step - loss: 0.4839 - binary_accuracy: 0.8256 - val_loss: 0.4634 - val_binary_accuracy: 0.8585
Epoch 2/3
33/33 [==============================] - 1s 45ms/step - loss: 0.1911 - binary_accuracy: 0.9183 - val_loss: 0.4075 - val_binary_accuracy: 0.8488
Epoch 3/3
33/33 [==============================] - 2s 46ms/step - loss: 0.1622 - binary_accuracy: 0.9293 - val_loss: 0.4120 - val_binary_accuracy: 0.8244


In [51]:
#==========================MY KNN MODEL===============================
model1 = KNeighborsClassifier()
model1.fit(x_train, y_train)

y_pred = model1.predict(x_test)
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.88      0.80      0.84       109
           1       0.79      0.88      0.83        96

    accuracy                           0.83       205
   macro avg       0.84      0.84      0.83       205
weighted avg       0.84      0.83      0.83       205

[[87 22]
 [12 84]]


0.8341463414634146

In [ ]:
# basically in the documentation you'll explain what activation functions are and the different types
#then you'll also explain optimizers and the different types
# explain that you used standardscaler for feature scaling
# standardscaler is important cos raw data has a huge difference between them that can affect the model
# so best thing is scaling the data before processing it
#in the second code i'll post i'll run the code without using the preprocessor to see how the accuracy
# of both models depriciated
# i'm sure you can explain confusion matrix and the other performance metrics
# for the deep learning model just explain the key features like optimizers, activation functions
# epochs, batch_size and the loss function
# loss function shows the distance between the current output and the expected output
# methodology just say you used a deep learning model first to train the model
# you'll also explain tensorflow and keras thats wah you used to construct the DLM
# the neural network is 4 layers that are densely connected(fully connected)
# note that the activation function you use for the output layer cant be same with other layers
# in this case i used sigmoid for the output layer
# you can also uyse softmax for the output layer
#if you have any question just ask me